<a href="https://colab.research.google.com/github/DManiscalco/MMA-Matchups/blob/main/MMA_Matchups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.window import Window

### Download the dataset and bring in the dataframe

In [2]:
# Download the dataset from kaggle
path = kagglehub.dataset_download('danmcinerney/mma-differentials-and-elo')

# Use $ to keep python variable in the terminal command
!ls $path  # make sure there are files in the path as we expect
!cp -r $path/* /content/  # move to /content folder

masterdataframe.csv  masterMLpublic.csv


In [3]:
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

In [4]:
# Read the CSV file
mma_df = spark.read.csv('/content/masterdataframe.csv', header=True, inferSchema=True)

In [5]:
# Show the first few rows of the DataFrame
# mma_df.show(10)

### Start preprocessing the data

In [6]:
## Drop any column that is a url:
# Get the list of column names where the first entry starts with 'http:' and drop them
first_row = mma_df.first()  # Get first row of data from dataframe
columns_with_http = [col for col in mma_df.columns if str(first_row[col]).startswith("http:")]
mma_df_reduced = mma_df.drop(*columns_with_http)  # Drop these columns

# Dropping 'time' col - it is the time that we run df and unrelated to the data
mma_df_reduced = mma_df_reduced.drop('time')

# 'age' col refers to age in 2021 so we calculate true age at time of fight
mma_df_reduced = mma_df_reduced.withColumn(
    'age',
    F.floor(F.months_between('date', 'dob') / 12)  # returns NULL if a col is NULL
)

In [7]:
# mma_df_reduced.show(5)

In [8]:
## Split the dataframe into a training and testing set based on specific fighters
## and the dates of their fights
# Proportions for training and testing sets
train_ratio = 0.7  # 70% for training
test_ratio = 0.3   # 30% for testing

# Calculate the total row count per fighter then cutoff counts based on proportions
row_counts = mma_df_reduced.groupBy('fighter').count().withColumnRenamed('count', 'total_rows')
cutoffs = row_counts.withColumn('train_cutoff', (F.col('total_rows') * train_ratio).cast('int'))
cutoffs = cutoffs.withColumn('test_cutoff', F.col('total_rows') - F.col('train_cutoff'))

# Join the cutoffs back to the original dataframe for reference
mma_df_reduced = mma_df_reduced.join(cutoffs.select('fighter', 'train_cutoff', 'test_cutoff'), on='fighter', how='left')

# Order rows by date within each person's group and assign row numbers
window = Window.partitionBy('fighter').orderBy('date')
mma_df_reduced = mma_df_reduced.withColumn('row_num', F.row_number().over(window))

# Create train and test dfs based on row numbers and cutoffs
train_df = mma_df_reduced.filter(F.col('row_num') <= F.col('train_cutoff')).drop('row_num', 'train_cutoff', 'test_cutoff')
test_df = mma_df_reduced.filter(F.col('row_num') > F.col('train_cutoff')).drop('row_num', 'train_cutoff', 'test_cutoff')